In [19]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import Callback

import time

In [20]:
#loading iris dataset
iris_data = load_iris() 


In [21]:
iris_data.data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [22]:
iris_data.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [23]:
iris_data.target.shape

(150,)

In [24]:
#isolating the features
x = iris_data.data
y_ = iris_data.target.reshape(-1, 1)

In [25]:
y_.shape

(150, 1)

In [26]:
#encoding data
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y_)

In [27]:
#splitting into train and test data
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)

In [28]:
#defining a function that calculates the change in mean weights at every epoch 
class WeightChangeLogger(Callback):
    def __init__(self):
        super(WeightChangeLogger, self).__init__()
        self.prev_weights = None

    def on_epoch_end(self, epoch, logs=None):
        current_weights = self.model.get_weights()
        if self.prev_weights is not None:
            changes = [np.mean(np.abs(current_weights[i] - self.prev_weights[i])) for i in range(len(current_weights))]
            print("Epoch {}: Average weight changes: {}".format(epoch+1, changes))
        self.prev_weights = current_weights


In [29]:
#defining model
model = Sequential()

model.add(Dense(10, input_shape=(4,), activation='relu', name='fc1'))
model.add(Dense(10, activation='relu', name='fc2'))
model.add(Dense(3, activation='softmax', name='output'))

optimizer = Adam(learning_rate=0.001)
start_time=time.time()
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc1 (Dense)                 (None, 10)                50        
                                                                 
 fc2 (Dense)                 (None, 10)                110       
                                                                 
 output (Dense)              (None, 3)                 33        
                                                                 
Total params: 193 (772.00 Byte)
Trainable params: 193 (772.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
weight_change_logger = WeightChangeLogger()


In [32]:
#callback to ewight logger function to visualize change
model.fit(train_x, train_y, verbose=2, batch_size=5, epochs=25, callbacks=[weight_change_logger])
end_time=time.time()

Epoch 1/25
24/24 - 1s - loss: 1.1766 - accuracy: 0.3000 - 897ms/epoch - 37ms/step
Epoch 2/25
Epoch 2: Average weight changes: [0.0056492835, 0.0049567306, 0.0050392766, 0.006746041, 0.005417859, 0.007571205]
24/24 - 0s - loss: 0.9512 - accuracy: 0.4583 - 40ms/epoch - 2ms/step
Epoch 3/25
Epoch 3: Average weight changes: [0.0048359465, 0.0028122398, 0.0036475756, 0.0031808268, 0.0033603122, 0.0036237112]
24/24 - 0s - loss: 0.8751 - accuracy: 0.7000 - 48ms/epoch - 2ms/step
Epoch 4/25
Epoch 4: Average weight changes: [0.0046393024, 0.0042840512, 0.0034141326, 0.0037406161, 0.0033185247, 0.003915678]
24/24 - 0s - loss: 0.8291 - accuracy: 0.7333 - 30ms/epoch - 1ms/step
Epoch 5/25
Epoch 5: Average weight changes: [0.004506409, 0.0034338683, 0.0033427859, 0.0028519516, 0.0037279115, 0.0025246998]
24/24 - 0s - loss: 0.7819 - accuracy: 0.7167 - 41ms/epoch - 2ms/step
Epoch 6/25
Epoch 6: Average weight changes: [0.0045159124, 0.003441599, 0.0030971062, 0.00310942, 0.003918359, 0.0031083387]
24/24 

In [33]:
test_loss, test_acc = model.evaluate(test_x, test_y)
print(test_acc)

1/1 [==============================] - 0s 204ms/step - loss: 0.2989 - accuracy: 0.9000
0.8999999761581421


In [34]:
#prinitng execution time
execution_time=end_time-start_time
print(execution_time)

2.0103540420532227


In [35]:
#printing the final array of weights
model.get_weights()

[array([[ 0.6237244 ,  0.0718639 ,  0.18101294,  0.3312774 , -0.0617429 ,
          0.22552104, -0.17106512,  0.6084671 , -0.315427  ,  0.28876907],
        [ 0.31440726, -0.21187618,  0.47017854, -0.0582916 , -0.22373766,
          0.24090272, -0.39080802, -0.11754625, -0.4586411 ,  0.10393736],
        [ 0.4780354 ,  0.19286993,  0.28782043, -0.08602691, -0.23530519,
         -0.10661801, -0.49158943, -0.34050366,  0.02035397,  0.16572593],
        [ 0.21968214,  0.8956691 ,  0.33599964,  0.73217076,  0.62166834,
         -0.825295  , -0.35605147, -0.02203594,  0.18551326, -0.5818465 ]],
       dtype=float32),
 array([-0.01357273, -0.11002245,  0.10052095, -0.08613148,  0.        ,
         0.21268281,  0.        ,  0.12605861,  0.        ,  0.01962733],
       dtype=float32),
 array([[-0.4650793 , -0.4913606 ,  0.19049166, -0.0924438 ,  0.0103519 ,
         -0.225649  ,  0.33945835,  0.02313901, -0.22725224,  0.20281713],
        [-0.11869834,  0.27928406, -0.6227722 ,  0.28568318, 

In [36]:
#calculating the total number of weights updated every epoch
array_3d=model.get_weights()
for i in range(len(array_3d)):
    print(array_3d[i].shape)

(4, 10)
(10,)
(10, 10)
(10,)
(10, 3)
(3,)
